# Tutorial: Train PyTorch Models

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
features, labels = datasets.load_breast_cancer(return_X_y=True)
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2)

In [3]:
features.shape, labels.shape

((569, 30), (569,))

In [4]:
import torch
import torch.nn as nn
import torch.utils.data as D
from kuma_utils.torch import TorchTrainer, EarlyStopping, TorchLogger
from kuma_utils.torch.model_zoo import TabularNet
from kuma_utils.metrics import AUC, Accuracy
from kuma_utils.utils import sigmoid

torch_xla not found.
nvidia apex not found.


In [5]:
train_ds = D.TensorDataset(
    torch.as_tensor(x_train).float(),
    torch.as_tensor(y_train.reshape(-1, 1)).float()
)
valid_ds = D.TensorDataset(
    torch.as_tensor(x_valid).float(),
    torch.as_tensor(y_valid.reshape(-1, 1)).float()
)
train_loader = D.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_loader = D.DataLoader(valid_ds, batch_size=64, shuffle=False)

In [6]:
model = TabularNet(in_features=x_train.shape[1], out_features=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

fit_params = {
    'loader': train_loader,
    'loader_valid': valid_loader,
    'criterion': criterion,
    'optimizer': optimizer,
    'scheduler': scheduler,
    'num_epochs': 100,
    'callbacks': [EarlyStopping(5, maximize=True, skip_epoch=10)],
    'eval_metric': AUC().torch,
    'monitor_metrics': [Accuracy().torch], 
    'logger': TorchLogger(
        'results/baseline/log.log', 
        log_items=['epoch', 'loss_valid', 'metric_train', 
                   'metric_valid', 'monitor_metrics_valid', 'earlystop']),
    # 'calibrate_model': True, 
    'snapshot_path': 'results/baseline/',
    # 'resume': True
}

TorchLogger created at 20/11/20:09:55:22


In [7]:
trn = TorchTrainer(model)
trn.train(**fit_params)

[Trainer] On cpu.
09:55:22 [  1/100] loss_valid=0.659751 | metric_train=0.556256 | metric_valid=0.694287 | monitor_metrics_valid=[0.359649] 
09:55:22 [  2/100] loss_valid=0.642436 | metric_train=0.539844 | metric_valid=0.963916 | monitor_metrics_valid=[0.252385] 
09:55:22 [  3/100] loss_valid=0.621979 | metric_train=0.578206 | metric_valid=0.944537 | monitor_metrics_valid=[0.283934] 
09:55:22 [  4/100] loss_valid=0.536184 | metric_train=0.593938 | metric_valid=0.890745 | monitor_metrics_valid=[0.283241] 
09:55:22 [  5/100] loss_valid=0.525523 | metric_train=0.609526 | metric_valid=0.930170 | monitor_metrics_valid=[0.255540] 
09:55:22 [  6/100] loss_valid=0.488632 | metric_train=0.740713 | metric_valid=0.861009 | monitor_metrics_valid=[0.454832] 
09:55:22 [  7/100] loss_valid=0.532457 | metric_train=0.676468 | metric_valid=0.938189 | monitor_metrics_valid=[0.255540] 
09:55:22 [  8/100] loss_valid=0.494024 | metric_train=0.744028 | metric_valid=0.816572 | monitor_metrics_valid=[0.509003]

In [8]:
trn.outoffold.shape

(114, 1)